In [1]:
import numpy as np
from squid_lab_quam.components.roots import SQuIDRoot1
import qm.qua as qua
from qm.simulate.interface import SimulationConfig
from quantum_control import AxisTracker, StreamManager

2024-07-22 16:04:29,668 - qm - INFO     - Starting session: a19bc59b-7a2b-415c-b0a3-fa1853e86a61


In [2]:
STATE_PATH = "N:\SCI-NBI-QDev\Jacob\quam_testing\state"

In [3]:
machine = SQuIDRoot1.load(STATE_PATH)

In [4]:
qubit = machine.qubits["q0"]
resonator = qubit.resonator

In [5]:
if_frequencies = np.linspace(-200e6, 200e6, 201)

In [6]:
resonator.channel.operations["readout"]

SquareReadoutPulse(length=1000, id=None, digital_marker='ON', amplitude=0.1, axis_angle=None, threshold=None, rus_exit_threshold=None, integration_weights=None, integration_weights_angle=0)

In [25]:
tracker = AxisTracker()
stream_manager = StreamManager()


with qua.program() as program:
    iq_stream = stream_manager.add_iq_stream("x")
    I, Q = qua.declare(qua.fixed), qua.declare(qua.fixed)
    i = qua.declare(int)
    # I_stream, Q_stream = qua.declare_stream(), qua.declare_stream()
    # with qua.for_(i, 0, i < len(if_frequencies), i + 1):
    #         qubit.measure(pulse_name="readout", qua_vars=(I, Q))
    #         qua.save(I, I_stream)
    #         qua.save(Q, Q_stream)
    with tracker.add_axis(qua.for_(i, 0, i < len(if_frequencies), i + 1), name="if_freq"):
        
        qubit.measure(pulse_name="readout", qua_vars=(I, Q))
        iq_stream.save(I, Q)

    with qua.stream_processing():
        # I_stream.save_all("I")
        # Q_stream.save_all("Q")
        iq_stream.process()


In [26]:
job = machine.execute(program)

2024-07-22 16:09:31,954 - qm - INFO     - Sending program to QOP for compilation
2024-07-22 16:09:32,061 - qm - INFO     - Executing program


In [27]:
job.result_handles.is_processing()

False

In [28]:
stream_manager.fetch_dataset(job)

<xarray.Dataset>
Dimensions:  (if_freq: 201)
Coordinates:
  * if_freq  (if_freq) int32 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199 200
Data variables:
    x        (if_freq) complex128 (0.016218718141317368+0.0033966414630413055...
Attributes:
    time_added_local:  2024-07-22T16:11:35+02:00
    time_added:        2024-07-22T14:11:35+00:00
    execution_time:    -121.906471
    job_id:            1719208397988
    job_status:        completed